In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease [18.1 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:13 http://security.ubuntu.com

In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-01-18 07:43:11--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.67MB/s    in 0.2s    

2023-01-18 07:43:11 (4.67 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [4]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Luggage_v1_00.tsv.gz" 

spark.sparkContext.addFile(url)
review_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Luggage_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True, timestampFormat="yyyy-mm-dd")

# Show DataFrame
review_data_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   40884699| R9CO86UUJCAW5|B00VGTN02Y|     786681372|Teenage Mutant Ni...|         Luggage|          3|            0|          0|   N|                Y|my review of this...|my review of this...|2015-01-31 00:08:00|
|         US|   23208852|R3PR8X6QGVJ8B1|B005KIWL0E|     618251799|Kenneth Cole Reac...| 

In [5]:
# Get the number of rows in the DataFrame.
print((review_data_df.count(), len(review_data_df.columns)))

(348657, 15)


In [6]:
# Drop null values, by default is any which means will drop any rows which has a NA
cleaned_df = review_data_df.dropna(how='any')
# Drop any duplicate
cleaned_df =  cleaned_df.dropDuplicates()
print((cleaned_df.count(), len(cleaned_df.columns)))

(348613, 15)


# Transform the Data

## Create the "review_id_table".

In [7]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_df = cleaned_df.select(['review_id','customer_id','product_id','product_parent','review_date'])
review_id_df.show()

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
| R5MPHGZNMQRNJ|   42610491|B00EAKKOKW|     393337306|2015-01-31 00:08:00|
|R2SCIZ2KIIW7G2|   22778458|B00VWKS05Y|     197290186|2015-01-31 00:08:00|
|R2Z98HWZHMV78N|    8405243|B00UQOUBV8|     960742333|2015-01-30 00:08:00|
|R1LSO9GG4P33VV|   27718120|B00BYFC4Z4|     506088001|2015-01-30 00:08:00|
| RS8HVEPCCM1V1|   14600611|B002YM70QO|     479181625|2015-01-30 00:08:00|
|R2CRLDSELLXESD|   44080043|B00JM2DLLC|      11240118|2015-01-30 00:08:00|
|R3ORL5J14E2IV7|   43765856|B00CPSWD8C|     677901073|2015-01-30 00:08:00|
|R2DC5GLVE7KPWC|   36190140|B000P8JBPC|      48200029|2015-01-29 00:08:00|
|R1D7LO4KL2XYNS|   34140095|B013EAIRPM|     677597111|2015-01-29 00:08:00|
|R3RE7EL02TIN1X|    9090247|B00MPSGZT0|     668538183|2015-01-28 00:08:00|
|R28LWDNNRDKV8E|   493897

## Create the "products" Table

In [8]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = cleaned_df.select(['product_id','product_title'])
products_df = products_df.distinct()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B005GUQCGK|Everest Gym Bag w...|
|B00M6O0WQK|Sprayground Unise...|
|B00UFFW5XK|Polare Italian Le...|
|B00J8O730E|Nicole Lee Shoppi...|
|B0028Y67PI|David King & Co. ...|
|B00DHCUWD8|PealRa Banana Lea...|
|B00O9QORAK|Luxebell Luggage ...|
|B0125Q648I|Shacke 2 x Metal ...|
|B004URKJY6|Small Orange Sol ...|
|B004R6B5MU|New Maxam 16 Inch...|
|B00FB4X9TY|Rockland Luggage ...|
|B00IDSDI1E|Kobwa(TM) Yellow ...|
|B00AE0NW5Q|Pacsafe Venturesa...|
|B00YNP622S|Great White Shark...|
|B00QF5Q7KG|Lewis N Clark Bal...|
|B0109ZA54Q|Passport Holder- ...|
|B00KYFPMWC|ECOSUSI Vintage C...|
|B005H4D4TI|CalPak Hollywood ...|
|B0053GFAJC|ShedRain Umbrella...|
|B00PY00AWY|Unisex Casual Gen...|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [9]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
customers_df = cleaned_df.groupby('customer_id').agg({'customer_id':"count"})
customers_df = customers_df.orderBy('count(customer_id)',ascending = False)
customers_df = customers_df.withColumnRenamed('count(customer_id)','customer_count')
customers_df = customers_df.dropna(how='any')
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   12640280|            63|
|   27493011|            31|
|   16600581|            31|
|   43856165|            29|
|   39789300|            26|
|   35461976|            25|
|   35938574|            24|
|    5329443|            23|
|   52228204|            23|
|   52955780|            23|
|   37244426|            22|
|   49266466|            21|
|   51712970|            19|
|     497832|            19|
|   47169566|            19|
|   20844007|            19|
|   51346302|            18|
|   10009912|            17|
|   14866545|            17|
|   11422115|            17|
+-----------+--------------+
only showing top 20 rows



## Create the "vine_table".

In [10]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df = cleaned_df.select(['review_id','star_rating','helpful_votes','total_votes','vine'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| R5MPHGZNMQRNJ|          5|            0|          0|   N|
|R2SCIZ2KIIW7G2|          5|            0|          0|   N|
|R2Z98HWZHMV78N|          5|            0|          0|   N|
|R1LSO9GG4P33VV|          5|            0|          0|   N|
| RS8HVEPCCM1V1|          4|            0|          0|   N|
|R2CRLDSELLXESD|          1|            1|          3|   N|
|R3ORL5J14E2IV7|          5|           27|         28|   N|
|R2DC5GLVE7KPWC|          5|            0|          0|   N|
|R1D7LO4KL2XYNS|          3|            1|          1|   N|
|R3RE7EL02TIN1X|          5|            0|          0|   N|
|R28LWDNNRDKV8E|          4|            0|          0|   N|
|R2IX2UD0OG88GA|          5|            0|          0|   N|
|R11910J20N35KD|          4|            2|          3|   N|
|R35RWEOCTVPYMH|          5|            

# Load

In [13]:
mode = "append"
jdbc_url="jdbc:postgresql://big-data-1.clo5tanjlfil.ap-southeast-2.rds.amazonaws.com:5432/amazon_review_part1_data1"
config = {"user":"postgres", "password": "qwertyui", "driver":"org.postgresql.Driver"}

In [15]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [16]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [17]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [18]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)